In [57]:
import time
import tweepy
import operator
import pandas as pd
from collections import defaultdict

from tweepy import OAuthHandler
from get_config import get_config



env = get_config()

consumer_key = env.get('CONSUMER_KEY')
consumer_secret = env.get('CONSUMER_SECRET')
access_token = env.get('ACCESS_TOKEN')
access_secret = env.get('ACCESS_TOKEN_SECRET')
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [59]:
# pull out 200 most recent tweets
# not sure what the page parameter does (but it does affect the output)
def one_search():
    return api.user_timeline(id="@ThisIsFusion",count=200,page=6,include_rts=True)

tweets = one_search()

In [60]:
# the most n retweets in the 200 recent tweets
def most_retweets(n):
    # to scale it later: filter out viral tweets by other users in fusion timeline
    # by checking .retweeted_status
    tweets.sort(key=operator.attrgetter('retweet_count'), reverse=True)
    return [(t.id, t.retweet_count) for t in tweets[:n]]

most = most_retweets(5)
print "The 5 most retweets\n"
print "      id          frequency"
for i in most:
    print i

The 5 most retweets

      id          frequency
(621405443135410176, 6539)
(621334501419954176, 113)
(621430450825887744, 96)
(621045435680059392, 80)
(620943905807290368, 68)


In [50]:
# first 100 retweet user id list for the nth most retweet
def retweet_users_of_a_tweet(tweet_id):
    retweets = api.retweets(tweet_id, 100)
    return [rt.user.id for rt in retweets]

# for debugging
def retweet_names_of_a_tweet(tweet_id):
    retweets = api.retweets(tweet_id, 100)
    return [rt.user.name for rt in retweets]

# get all followers of a user
def followers(user_id):
    ids = []
    for page in tweepy.Cursor(api.followers_ids, id=user_id).pages():
        ids.extend(page)
        time.sleep(60)

    return ids
    
# get all following of a user
def followings(user_id):
    ids = []
    for page in tweepy.Cursor(api.friends_ids, id=user_id).pages():
        ids.extend(page)
        time.sleep(60)

    return ids

This is the example of how to all call the above functions

In [89]:
# print retweet_users_of_tweet(616115346622296064)
# There might be some limitation because of twitter API
%time
a = followers(2801891267)
print len(a)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 10 µs
1365


example of getting the network dataframe from an about 100-times retweet

key is to contain the first retweet as a root

example retweet(id=621430450825887744) has 96 retweets

dataframe example:

| a       | b           | a_follow_b  |  b_follow_a|
| ------------- |:-------------:| -----:| -----:|
| user_1      | user_x | 1  |0
| user_2      | user_y      |0   |1
| ...      | ...      |...    | ...
| user_n | user_z     |1    |1

In [85]:
rt_users = retweet_users_of_a_tweet(621430450825887744)
print api.get_status(621430450825887744).text
print api.get_status(621430450825887744).created_at
print 'Of 96 retweets, %d are public' % len(rt_users)

# from the most to least recent
# print retweet_names_of_a_tweet(621430450825887744)
retweet_users = retweet_users_of_a_tweet(621430450825887744)

net = defaultdict(list)
n = 0
for user in retweet_users:
    retweet_users.pop(retweet_users.index(user))
    n = 0
    for another in retweet_users:
        if api.show_friendship(source_id=user,target_id=another)[0].followed_by:
            net['a'].append(user)
            net['b'].append(another)
            net['a_follow_b'].append(1)
            if api.show_friendship(source_id=user,target_id=another)[0].following:
                net['b_follow_a'].append(1)
            else:
                net['b_follow_a'].append(0)
        elif api.show_friendship(source_id=user,target_id=another)[0].following:
            net['a'].append(user)
            net['b'].append(another)
            net['a_follow_b'].append(0)
            net['b_follow_a'].append(1)
        print 'finished %d friendship lookup for one user' % n
        time.sleep(1)
    n += 1
    print 'finished %d of 92 users' % n
    
df = pd.DataFrame(net)
df.to_csv('net.csv', sep='\t', encoding='utf-8')

This Vine from the Harvard Innovation Lab perfectly shows how much we now rely on apps https://t.co/ROa7MgGQOe via @TheNextWeb
2015-07-15 21:25:39
Of 96 retweets, 92 are public


TweepError: [{u'message': u'Rate limit exceeded', u'code': 88}]